In [8]:
import pandas as pd
import numpy as np
import os
import requests 
import json
apiKey='0ce9de199a3449a8a43b70421678e0ae'
base_url = 'https://api.geoapify.com/v1/geocode/search?'

In [6]:
def GetPopulationData_1(csv_file):
    year = csv_file.split(".")[0][-4:]
    df = pd.read_csv(csv_file)
    df = df[["NAME","S0101_C01_001E"]]
    df.rename(columns={"NAME":"zip_code","S0101_C01_001E":"population"},inplace=True)
    df = df.iloc[1:]
    df["population"] = df["population"].astype(int)
    df.to_csv(f"output_first_part/{year}.csv",index=False)


def GetPopulationData_2(csv_file):
    year = csv_file.split(".")[0][-4:]
    df = pd.read_csv(csv_file)
    cols =  df.columns[df.columns.str.contains("Total!!Estimate")]
    df = df[cols]
    df.rename(columns= lambda x: x.split("!!")[0],inplace=True)
    df = df.iloc[0]
    df = df.apply(lambda x: x.replace(",","")).astype(int)
    frame = {"zip_code":df.index.values,"population":df.values}
    final_df = pd.DataFrame(frame)
    final_df.to_csv(f"output_second_part/{year}.csv",index=False)



In [7]:
#Getting second_part csv files paths
data_second_part = "data/second_part/"
csv_paths_2 = os.listdir(data_second_part)
for csv_file in csv_paths_2:
    csv_file_path = data_second_part  +csv_file
    GetPopulationData_2(csv_file_path)


OSError: Cannot save file into a non-existent directory: 'output_second_part'

In [10]:
import pandas as pd

files = ['2011.csv', '2012.csv', '2013.csv', '2014.csv','2015.csv','2016.csv','2017.csv','2018.csv']

combined_df = pd.DataFrame()
folder_name= "final_output"

for file in files:
    df = pd.read_csv(folder_name+"/"+file)
    year = int(file.split('.')[0])  
    df['year'] = year
    combined_df = pd.concat([combined_df, df], ignore_index=True)
combined_df=combined_df[['year','population','zip_code']]
combined_df['zip_code'] = combined_df["zip_code"].str.slice(5)
lat = []
lon = []
for code in combined_df['zip_code']:
    url = base_url+'text='+code+'&lang=en&limit=10&type=postcode&filter=countrycode:us&&format=json&apiKey='+apiKey
    response = requests.get(url).json()
    lat.append(response['results'][0]['lat'])
    lon.append(response['results'][0]['lon'])
combined_df['lat'] = lat 
combined_df['lon'] = lon  
combined_df.to_csv("final.csv",index=False)
combined_df

,year,population,zip_code,lat,lon
0,2011,16585,99501,61.216700,-149.887149
1,2011,24061,99502,61.147457,-150.017931
2,2011,13604,99503,61.188159,-149.888929
3,2011,39662,99504,61.203789,-149.738415
4,2011,6859,99505,61.269116,-149.639297
...,...,...,...,...,...
2331,2018,15,99923,55.918430,-130.026980
2332,2018,927,99925,55.554011,-133.077586
2333,2018,1635,99926,55.124316,-131.575660
2334,2018,38,99927,56.239062,-133.457924


In [11]:
#Concatenating both parts
years = range(2011,2019)
for y in years:
    df1 = pd.read_csv("output_first_part/{}.csv".format(y))
    df2 = pd.read_csv("output_second_part/{}.csv".format(y))
    df = pd.concat([df1,df2])
    df.to_csv("final_output/{}.csv".format(y),index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'output_first_part/2011.csv'

In [4]:
data_files = "/Users/hanenzeddini/Desktop/UCI_Bootcamp/Project_3/Project-3-grp-6/Project_3/final_output/"
cvs_paths = os.listdir(data_files)
cvs_paths

['2018.csv',
 '2015.csv',
 '2014.csv',
 '2016.csv',
 '2017.csv',
 '2013.csv',
 '2012.csv',
 '2011.csv']